In [34]:
# for ID signature
# caculation right repeat & microhomology for >=2bp deletion

import os,sys,pysam,gzip,collections
from numpy import median


def ID_repeat(input_line,ref_fasta):
    input_split = input_line.split('\t')
    input_chr = input_split[0]
    input_pos = long(input_split[1])
    
    input_ref = input_split[3]
    input_alt = input_split[4]
    
    if len(input_ref) == 1: # for small insertion
        ru = input_alt[1:]
        rc = 0
        
        while rc<5:
            if ref_fasta.fetch(input_chr,input_pos+len(ru)*rc,input_pos+len(ru)*(rc+1)) == ru:
                rc+=1
                #print ref_fasta.fetch(input_chr,input_pos+len(ru)*rc,input_pos+len(ru)*(rc+1))
            else:
                break
        #print ref_fasta.fetch(input_chr,input_pos-1,input_pos) + 'qwe'
        return ['insertion',str(ru),str(rc),'noMH']

    else: # for deletion    
        if len(input_ref) == 2:
            ru = input_ref[1:]
            rc = 0
            
            while rc<5:
                if ref_fasta.fetch(input_chr,input_pos+len(ru)*(rc+1),input_pos+len(ru)*(rc+2)) == ru:
                    rc+=1
                    #print ref_fasta.fetch(input_chr,input_pos+len(ru)*rc,input_pos+len(ru)*(rc+1))
                else:
                    break
            #print ref_fasta.fetch(input_chr,input_pos-2,input_pos) + 'qwe'
            return ['deletion',str(ru),str(rc+1),'noMH']            
            
            
        else:
            #print input_line
            ru = input_ref[1:]
            rc = 0
            #print ru
            #print ref_fasta.fetch(input_chr,input_pos+len(ru)*(rc+1),input_pos+len(ru)*(rc+2))
            
            if ref_fasta.fetch(input_chr,input_pos+len(ru)*(rc+1),input_pos+len(ru)*(rc+2)) == ru:
                while rc<5:
                    if ref_fasta.fetch(input_chr,input_pos+len(ru)*(rc+1),input_pos+len(ru)*(rc+2)) == ru:
                        rc+=1
                        #print ref_fasta.fetch(input_chr,input_pos+len(ru)*rc,input_pos+len(ru)*(rc+1))
                    else:
                        break
                #print ref_fasta.fetch(input_chr,input_pos-2,input_pos) + 'qwe'
                return ['deletion',str(ru),str(rc+1),'noMH']
            else: # microhomology check!
                rt_mh = 0
                lt_mh = 0
                for i in range(0,len(ru)):
                    if ref_fasta.fetch(input_chr,input_pos+len(ru),input_pos+len(ru)+1+i) == input_ref[1:1+1+i]:
                        rt_mh+=1
                    else:
                        break
                for j in range(0,len(ru)):
                    if ref_fasta.fetch(input_chr,input_pos-1-j,input_pos) == input_ref[-1-j:]:
                        lt_mh+=1
                    else:
                        break                                            
                if rt_mh == 0 & lt_mh ==0:
                    return ['deletion',str(ru),str(rc+1),'noMH']
                else:
                    #print input_line
                    return ['deletion',str(ru),'0',str(max(rt_mh,lt_mh))]
        
        
        
    
#start
temp_list = os.listdir("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse")
file_list = []
for ll in temp_list:
    if '.stricttrue.vcf' in ll:
        file_list.append(ll)

#file_list.append("")
print len(file_list)
#file_list = ["hs_study5_colon_control_HCNor1_SC5_2nd.true.clonal.vcf","hs_study5_colon_control_HCNor1_SC4_2nd.true.clonal.vcf"]

for input_fn in file_list:
    print input_fn
    #chr_pos_list=[]
    input_file = open("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/" + input_fn)
    output_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/" + input_fn.replace(".gz","").replace(".stricttrue.vcf",".IDsig1-10bp.strict.vcf"),"w")
    input_line = input_file.readline().strip()
    while input_line[0:2] =='##':
        input_line = input_file.readline().strip()
    output_file.write(input_line+'\tID_signature_type;repeat_unit;repat_count;microhomology\n')
    input_line = input_file.readline().strip()
    
    mm=0
    while input_line:
        #input_ref=sys.argv[1]
        input_ref = 'mm10'
        if input_ref == 'mm10':
            ref_fa='/home/users/jhyouk/99_reference/mouse/mm10/GRCm38.fa'
        elif input_ref == 'hg19':
            ref_fa='/home/users/jhyouk/99_reference/human/GRCh37/human_g1k_v37.fasta'
        r_file=pysam.FastaFile(ref_fa)

        #chr_pos_list.append([input_line.split('\t')[0],input_line.split('\t')[1]])
        if len(input_line.split('\t')[3]) > 11 or len(input_line.split('\t')[4]) > 11 :
            'blank'
        else:
            ID_list = ID_repeat(input_line,r_file)
            output_file.write(input_line + '\t' + ';'.join(ID_list) + '\n')

        input_line = input_file.readline().strip()


    output_file.close()
    
print 'THE END'
    

208
S1-4Gy-2.clonal.stricttrue.vcf
S1N2P16_2-3.stricttrue.vcf
B3-0_P19_0Gy-1.stricttrue.vcf
mm_study2_pancreas_sham_C3SO1SO1_normoxia_SO1.clonal.stricttrue.vcf
mm_study4_stomach_IR_SO1.stricttrue.vcf
mm_study4_fallopian_IR_SO2.stricttrue.vcf
mm_study3_pancreas_Lowdose_SO2.stricttrue.vcf
B3-0_P19_4Gy-2.stricttrue.vcf
S1N2P15_1-1.clonal.stricttrue.vcf
S1N2P16_2-5.clonal.stricttrue.vcf
Liver_C3SO1.stricttrue.vcf
S1N2P15_4-1.stricttrue.vcf
S1-2Gy-1.clonal.stricttrue.vcf
mm_study4_pancreas_sham_SO1.stricttrue.vcf
B3-0_2Gy-3.stricttrue.vcf
S1-1Gy-1.stricttrue.vcf
S1N2P15_2-8.clonal.stricttrue.vcf
mm_study2_liver_IR_8Gy_SO2.stricttrue.vcf
mm_study2_pancreas_8by1Gy_SO4.clonal.stricttrue.vcf
Panc_2Gy2_SO2.clonal.stricttrue.vcf
mm_study4_stomach_IR_SO2.stricttrue.vcf
mm_study4_stomach_IR_SO3.stricttrue.vcf
S1N2P15_0-4.clonal.stricttrue.vcf
S1N2P15_1-1.stricttrue.vcf
8Gy_2_S01_S.clonal.stricttrue.vcf
mm_study4_pancreas_IR_SO3.stricttrue.vcf
S1N2P15_0-5.clonal.stricttrue.vcf
S1P13_0-5.clonal.stric

In [35]:
#83 context

#'1bp_del'
#'1bp_ins'
#'2bp_del_repeat'
#'2bp_ins'
#'2bp_del_micohomology'

#start
temp_list = os.listdir("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/")
file_list = []
for ll in temp_list:
    if '.IDsig1-10bp.strict.vcf' in ll:
        file_list.append(ll)

        
print len(file_list)
#file_list = ["hs_study5_colon_control_HCNor1_SC4_2nd.true.clonal.IDsig1-10bp.vcf","hs_study5_colon_control_HCNor1_SC5_2nd.true.clonal.IDsig1-10bp.vcf","hs_study5_colon_control_HCNor1_SC5_2nd.true.IDsig1-10bp.vcf","hs_study5_colon_control_HCNor1_SC4_2nd.true.IDsig1-10bp.vcf",]

output_file = file("mouse_organoid_strict_200122.txt","w")
head_list = ['ID','Dose','d1c1', 'd1c2', 'd1c3', 'd1c4', 'd1c5', 'd1c6', 'd1t0', 'd1t1', 'd1t2', 'd1t3', 'd1t4', 'd1t5', 'i1c1', 'i1c2', 'i1c3', 'i1c4', 'i1c5', 'i1c6', 'i1t0', 'i1t1', 'i1t2', 'i1t3', 'i1t4', 'i1t5', 'd22_1', 'd22_2', 'd22_3', 'd22_4', 'd22_5', 'd22_6', 'd23_1', 'd23_2', 'd23_3', 'd23_4', 'd23_5', 'd23_6', 'd24_1', 'd24_2', 'd24_3', 'd24_4', 'd24_5', 'd24_6', 'd25_1', 'd25_2', 'd25_3', 'd25_4', 'd25_5', 'd25_6', 'i22_0', 'i22_1', 'i22_2', 'i22_3', 'i22_4', 'i22_5', 'i23_0', 'i23_1', 'i23_2', 'i23_3', 'i23_4', 'i23_5', 'i24_0', 'i24_1', 'i24_2', 'i24_3', 'i24_4', 'i24_5', 'i25_0', 'i25_1', 'i25_2', 'i25_3', 'i25_4', 'i25_5', 'mh2_1', 'mh3_1', 'mh3_2', 'mh4_1', 'mh4_2', 'mh4_3', 'mh5_1', 'mh5_2', 'mh5_3', 'mh5_4', 'mh5_5']
output_file.write('\t'.join(head_list) + "\n")

info_file = file("00_9_mouse_human_191010.txt")
info_line = info_file.readline().strip()
info_dose = {}
while info_line:
    info_split = info_line.split('\t')
    info_dose[info_split[0].replace("_retry","").replace("_2nd","")] = info_split[1]
    info_line = info_file.readline().strip()

#temp_info = ['PD8622a_indel.true.IDsig1-10bp.vcf', 'S15_40859_17_T_new.true.IDsig1-10bp.vcf', '18_1277T.true.IDsig1-10bp.vcf', 'PD7188a_indel.true.IDsig1-10bp.vcf', 'PD9056a_indel.true.IDsig1-10bp.vcf', 'PD7191a_indel.true.IDsig1-10bp.vcf', '18_1360T.true.IDsig1-10bp.vcf', '19-1546T.true.IDsig1-10bp.vcf', '19-1621T.true.IDsig1-10bp.vcf', 'S15_40859_T.true.IDsig1-10bp.vcf', 'PD13489a_indel.true.IDsig1-10bp.vcf', 'PD8623a_indel.true.IDsig1-10bp.vcf', '15-1494T_mem.true.IDsig1-10bp.vcf', 'PD9972a_indel.true.IDsig1-10bp.vcf', 'S17_11918T_re.true.IDsig1-10bp.vcf', 'PD7530a_indel.true.IDsig1-10bp.vcf', 'S17_50179_T.true.IDsig1-10bp.vcf', 'S12_10269_T.true.IDsig1-10bp.vcf', '18-2419T.true.IDsig1-10bp.vcf', 'PD7192a_indel.true.IDsig1-10bp.vcf', 'PD8618a_indel.true.IDsig1-10bp.vcf', 'S14_54949_T.true.IDsig1-10bp.vcf', '15-495-T.true.IDsig1-10bp.vcf', 'S17_11918T.true.IDsig1-10bp.vcf', 'S160041844_T_new.true.IDsig1-10bp.vcf', 'PD7189a_indel.true.IDsig1-10bp.vcf', '15-1034-T.true.IDsig1-10bp.vcf', 'PD7190a_indel.true.IDsig1-10bp.vcf']
#temp_dose = '0'

for input_fn in file_list:   
    print input_fn
    input_file = file("mouse/" + input_fn)
    
    del_1bp = []
    ins_1bp = []
    del_2bp = []
    ins_2bp = []
    del_mh = []

    for i in range(0,12):
        del_1bp.append(0)
        ins_1bp.append(0)
    for i in range(0,24):    
        del_2bp.append(0)
        ins_2bp.append(0)
    for i in range(0,11):
        del_mh.append(0)

    input_line = input_file.readline().strip()
    input_line = input_file.readline().strip()

    while input_line:
        #try:
        input_info = input_line.split('\t')[-1] ####
        #except:
        #    input_info = input_line.split('\t')[35] ####
            
        info_split = input_info.split(';')

        if len(info_split[1]) == 1:
            if info_split[0] == 'deletion':
                homopolymer=min(int(info_split[2]),6)
                if info_split[1] == 'C' or info_split[1] == 'G':
                    del_1bp[0+homopolymer-1] += 1          
                else:
                    del_1bp[6+homopolymer-1] += 1  
            else:
                homopolymer=min(int(info_split[2]),5)
                if info_split[1] == 'C' or info_split[1] == 'G':
                    ins_1bp[0+homopolymer]+=1          
                else:
                    ins_1bp[6+homopolymer]+=1
        elif info_split[0] == 'insertion':
            ins_length = min(len(info_split[1]),5)
            homopolymer=min(int(info_split[2]),5)
            ins_2bp[6*(ins_length-2)+homopolymer]+=1

        else:
            if info_split[3] == 'noMH':
                del_length = min(len(info_split[1]),5)
                homopolymer=min(int(info_split[2]),6)        
                del_2bp[6*(del_length-2) + homopolymer-1] += 1
            else:
                del_length = min(len(info_split[1]),5)
                mh = min(int(info_split[3]),5)
                if del_length == 2:
                    del_mh[0] += 1
                elif del_length == 3:
                    del_mh[mh] += 1
                elif del_length == 4:
                    del_mh[3-1+mh] += 1
                else:
                    del_mh[6-1+mh] += 1
        input_line = input_file.readline().strip()

    temp_total = ""
    for i in del_1bp:
        temp_total = temp_total + '\t' + str(i)
    for i in ins_1bp:
        temp_total = temp_total + '\t' + str(i)
    for i in del_2bp:
        temp_total = temp_total + '\t' + str(i)
    for i in ins_2bp:
        temp_total = temp_total + '\t' + str(i)
    for i in del_mh:
        temp_total = temp_total + '\t' + str(i)
    #if 'PCAWG' in input_fn:
    #    temp_dose = '0'
    #else:
    #    temp_dose = '60'
    output_file.write(input_fn.split('.IDsig1-10bp')[0] +'\t'+info_dose[input_fn.split('.')[0]] +temp_total + '\n')        
    #output_file.write(input_fn.split('.')[0] +'\t' + temp_dose + temp_total + '\n')

output_file.close()  
print 'THE END'

208
S1N2P15_1-5a.clonal.IDsig1-10bp.strict.vcf
mm_study4_breast_IR_SO1.IDsig1-10bp.strict.vcf
S1N2P16_2-3.IDsig1-10bp.strict.vcf
S1N2P15_2-2.IDsig1-10bp.strict.vcf
mm_study2_liver_8by1Gy_SO4.IDsig1-10bp.strict.vcf
mm_study4_SI_sham_SO3.IDsig1-10bp.strict.vcf
S1-8Gy-1.IDsig1-10bp.strict.vcf
S1N2P16_4-7.IDsig1-10bp.strict.vcf
S1N2P15_1-4a.clonal.IDsig1-10bp.strict.vcf
B3S100.IDsig1-10bp.strict.vcf
S1-2Gy-2.clonal.IDsig1-10bp.strict.vcf
N1-S1.clonal.IDsig1-10bp.strict.vcf
S1N2P15_1-1.IDsig1-10bp.strict.vcf
S1P13_0-5.IDsig1-10bp.strict.vcf
mm_study4_stomach_IR_SO2.IDsig1-10bp.strict.vcf
S1P13_0-4.IDsig1-10bp.strict.vcf
S1-1Gy-2.clonal.IDsig1-10bp.strict.vcf
S1N2P15_0-1.IDsig1-10bp.strict.vcf
S1N2P15_0-3.IDsig1-10bp.strict.vcf
S1N2P15_2-8.clonal.IDsig1-10bp.strict.vcf
2Gy_2_S03_S.IDsig1-10bp.strict.vcf
mm_study4_fallopian_IR_SO2.IDsig1-10bp.strict.vcf
Liver_20Gy1_SO2.IDsig1-10bp.strict.vcf
8Gy_2_S01_S.IDsig1-10bp.strict.vcf
B3-0_P19_0Gy-1.IDsig1-10bp.strict.vcf
mm_study2_pancreas_8by1Gy_SO4

In [13]:
#83 context in nonrepetitive regions

#'1bp_del'
#'1bp_ins'
#'2bp_del_repeat'
#'2bp_ins'
#'2bp_del_micohomology'

#start
temp_list = os.listdir("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/human")
file_list = []
for ll in temp_list:
    if '.true.clonal.IDsig1-10bp.vcf' in ll:
        file_list.append(ll)
        
print len(file_list)
#file_list = ["S1N2P16_4-4.true.plusdelly.vcf"]
output_file = file("IDsig2_human_clonal_nonrepeat.txt","w")
head_list = ['ID','Dose','d1c1', 'd1c2', 'd1c3', 'd1c4', 'd1c5', 'd1c6', 'd1t0', 'd1t1', 'd1t2', 'd1t3', 'd1t4', 'd1t5', 'i1c1', 'i1c2', 'i1c3', 'i1c4', 'i1c5', 'i1c6', 'i1t0', 'i1t1', 'i1t2', 'i1t3', 'i1t4', 'i1t5', 'd22_1', 'd22_2', 'd22_3', 'd22_4', 'd22_5', 'd22_6', 'd23_1', 'd23_2', 'd23_3', 'd23_4', 'd23_5', 'd23_6', 'd24_1', 'd24_2', 'd24_3', 'd24_4', 'd24_5', 'd24_6', 'd25_1', 'd25_2', 'd25_3', 'd25_4', 'd25_5', 'd25_6', 'i22_0', 'i22_1', 'i22_2', 'i22_3', 'i22_4', 'i22_5', 'i23_0', 'i23_1', 'i23_2', 'i23_3', 'i23_4', 'i23_5', 'i24_0', 'i24_1', 'i24_2', 'i24_3', 'i24_4', 'i24_5', 'i25_0', 'i25_1', 'i25_2', 'i25_3', 'i25_4', 'i25_5', 'mh2_1', 'mh3_1', 'mh3_2', 'mh4_1', 'mh4_2', 'mh4_3', 'mh5_1', 'mh5_2', 'mh5_3', 'mh5_4', 'mh5_5']
output_file.write('\t'.join(head_list) + "\n")

info_file = file("00_9_mouse_human_191010.txt")
info_line = info_file.readline().strip()
info_dose = {}
while info_line:
    info_split = info_line.split('\t')
    info_dose[info_split[0].replace("_retry","")] = info_split[1]
    info_line = info_file.readline().strip()

repeat_count = 5 ### define same to my analysis

for input_fn in file_list:   
    print input_fn
    input_file = file("human/" + input_fn)
    
    del_1bp = []
    ins_1bp = []
    del_2bp = []
    ins_2bp = []
    del_mh = []

    for i in range(0,12):
        del_1bp.append(0)
        ins_1bp.append(0)
    for i in range(0,24):    
        del_2bp.append(0)
        ins_2bp.append(0)
    for i in range(0,11):
        del_mh.append(0)

    input_line = input_file.readline().strip()
    input_line = input_file.readline().strip()

    while input_line:
        input_split = input_line.split('\t')
        input_lru = int(input_split[29].split(';')[-4])
        input_rru = int(input_split[29].split(';')[-3])  
        if input_lru < repeat_count and input_rru < repeat_count:
            'blank'
        else:
            input_line = input_file.readline().strip()
            continue
            
        try:
            input_info = input_line.split('\t')[42]
        except:
            input_info = input_line.split('\t')[35]
            
        info_split = input_info.split(';')

        if len(info_split[1]) == 1:
            if info_split[0] == 'deletion':
                homopolymer=min(int(info_split[2]),6)
                if info_split[1] == 'C' or info_split[1] == 'G':
                    del_1bp[0+homopolymer-1] += 1          
                else:
                    del_1bp[6+homopolymer-1] += 1  
            else:
                homopolymer=min(int(info_split[2]),5)
                if info_split[1] == 'C' or info_split[1] == 'G':
                    ins_1bp[0+homopolymer]+=1          
                else:
                    ins_1bp[6+homopolymer]+=1
        elif info_split[0] == 'insertion':
            ins_length = min(len(info_split[1]),5)
            homopolymer=min(int(info_split[2]),5)
            ins_2bp[6*(ins_length-2)+homopolymer]+=1

        else:
            if info_split[3] == 'noMH':
                del_length = min(len(info_split[1]),5)
                homopolymer=min(int(info_split[2]),5)        
                del_2bp[6*(del_length-2) + homopolymer-1] += 1
            else:
                del_length = min(len(info_split[1]),5)
                mh = min(int(info_split[3]),5)
                if del_length == 2:
                    del_mh[0] += 1
                elif del_length == 3:
                    del_mh[mh] += 1
                elif del_length == 4:
                    del_mh[3-1+mh] += 1
                else:
                    del_mh[6-1+mh] += 1
        input_line = input_file.readline().strip()

    temp_total = ""
    for i in del_1bp:
        temp_total = temp_total + '\t' + str(i)
    for i in ins_1bp:
        temp_total = temp_total + '\t' + str(i)
    for i in del_2bp:
        temp_total = temp_total + '\t' + str(i)
    for i in ins_2bp:
        temp_total = temp_total + '\t' + str(i)
    for i in del_mh:
        temp_total = temp_total + '\t' + str(i)
        
    output_file.write(input_fn.split('.true.')[0] +'\t'+info_dose[input_fn.split('.true.')[0]] +temp_total + '\n')        

output_file.close()  
print 'THE END'

15
HBIR1-1.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_control_HCNor2_SC1.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_control_HCNor2_SC3.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_IR_HCIR1_SC6.true.clonal.IDsig1-10bp.vcf
HBIR2-1.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_control_HCNor1_SC4.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_control_HCNor1_SC5.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_control_HCNor2_SC5.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_IR_HCIR1_SC1.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_control_HCNor2_SC4.true.clonal.IDsig1-10bp.vcf
HBIR1-2.true.clonal.IDsig1-10bp.vcf
HBIR2-4.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_control_HCNor2_SC6.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_control_HCNor2_SC2.true.clonal.IDsig1-10bp.vcf
hs_study5_colon_IR_HCIR1_SC2.true.clonal.IDsig1-10bp.vcf
THE END


In [92]:
input_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/human/human_tumor/RIS_naturecom_indel.txt")
input_line = input_file.readline().strip()
id_list = []
while input_line:
    input_split = input_line.split('\t')
    if input_split[0] in id_list:
        output_file.write('\t'.join(input_split[1:3])+"\t.\t"+'\t'.join(input_split[3:5])+'\n')       
    else:
        print input_split[0]
        if len(id_list)>0:
            output_file.close()
        id_list.append(input_split[0])
        output_file=file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/human/human_tumor/%s_indel.true.vcf" % input_split[0],"w")
        output_file.write("#CHROM\tPOS\tRS\tRef\tAlt\n")
        output_file.write('\t'.join(input_split[1:3])+"\t.\t"+'\t'.join(input_split[3:5])+'\n')
    input_line = input_file.readline().strip()
output_file.close()
print 'THE END'
    

PD13489a
PD7188a
PD7189a
PD7190a
PD7191a
PD7192a
PD7530a
PD8618a
PD8622a
PD8623a
PD9056a
PD9972a
THE END


In [90]:
input_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/human/human_tumor/RIS_naturecom_indel.txt")
input_line = input_file.readline().strip()
id_list = []
while input_line:
    input_split = input_line.split('\t')
    print input_split[1:3]
    break
    input_line = input_file.readline().strip()

['1', '7973792']


In [105]:
temp_list = os.listdir("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/human/human_tumor/")
file_list = []
for ll in temp_list:
    if 'PD' in ll or 'T.true' in ll or 're.true' in ll or 'mem.true' in ll or 'new.true' in ll:
        if 'IDsig1-10bp.vcf' in ll:
            file_list.append(ll)
print file_list
print len(file_list)

['PD8622a_indel.true.IDsig1-10bp.vcf', 'S15_40859_17_T_new.true.IDsig1-10bp.vcf', '18_1277T.true.IDsig1-10bp.vcf', 'PD7188a_indel.true.IDsig1-10bp.vcf', 'PD9056a_indel.true.IDsig1-10bp.vcf', 'PD7191a_indel.true.IDsig1-10bp.vcf', '18_1360T.true.IDsig1-10bp.vcf', '19-1546T.true.IDsig1-10bp.vcf', '19-1621T.true.IDsig1-10bp.vcf', 'S15_40859_T.true.IDsig1-10bp.vcf', 'PD13489a_indel.true.IDsig1-10bp.vcf', 'PD8623a_indel.true.IDsig1-10bp.vcf', '15-1494T_mem.true.IDsig1-10bp.vcf', 'PD9972a_indel.true.IDsig1-10bp.vcf', 'S17_11918T_re.true.IDsig1-10bp.vcf', 'PD7530a_indel.true.IDsig1-10bp.vcf', 'S17_50179_T.true.IDsig1-10bp.vcf', 'S12_10269_T.true.IDsig1-10bp.vcf', '18-2419T.true.IDsig1-10bp.vcf', 'PD7192a_indel.true.IDsig1-10bp.vcf', 'PD8618a_indel.true.IDsig1-10bp.vcf', 'S14_54949_T.true.IDsig1-10bp.vcf', '15-495-T.true.IDsig1-10bp.vcf', 'S17_11918T.true.IDsig1-10bp.vcf', 'S160041844_T_new.true.IDsig1-10bp.vcf', 'PD7189a_indel.true.IDsig1-10bp.vcf', '15-1034-T.true.IDsig1-10bp.vcf', 'PD7190a_i

In [148]:
### RIS_SNUH indel strict selection criteria

import math

#ID_list = ['15-495-T', '15-1494T_mem', '15-1034-T', '18_1360T', '19-1621T', '18-2419T', '18_1277T', 'S160041844_T_new', '19-1546T', 'S14_54949_T', 'S15_40859_T', 'S17_11918T', 'S17_50179_T', 'S15_40859_17_T_new', 'S17_11918T_re', 'S12_10269_T']
#ID_list = ['HBIR2-1.true.vcf', 'hs_study5_colon_control_HCNor2_SC6.true.vcf', 'hs_study5_colon_control_HCNor1_SC5.true.vcf', 'hs_study5_colon_control_HCNor1_SC4.true.vcf', 'hs_study5_colon_control_HCNor2_SC4.true.vcf', 'hs_study5_colon_IR_HCIR1_SC2.true.vcf', 'hs_study5_colon_control_HCNor2_SC3.true.vcf', 'hs_study5_colon_IR_HCIR1_SC6.true.vcf', 'hs_study5_colon_control_HCNor2_SC1.true.vcf', 'hs_study5_colon_control_HCNor2_SC5.true.vcf', 'HBIR1-1.true.vcf', 'HBIR1-2.true.vcf', 'hs_study5_colon_IR_HCIR1_SC1.true.vcf', 'hs_study5_colon_control_HCNor2_SC2.true.vcf', 'HBIR2-4.true.vcf', 'HBIR2-1.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC6.true.clonal.vcf', 'hs_study5_colon_control_HCNor1_SC5.true.clonal.vcf', 'hs_study5_colon_control_HCNor1_SC4.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC4.true.clonal.vcf', 'hs_study5_colon_IR_HCIR1_SC2.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC3.true.clonal.vcf', 'hs_study5_colon_IR_HCIR1_SC6.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC1.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC5.true.clonal.vcf', 'HBIR1-1.true.clonal.vcf', 'HBIR1-2.true.clonal.vcf', 'hs_study5_colon_IR_HCIR1_SC1.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC2.true.clonal.vcf', 'HBIR2-4.true.clonal.vcf']




print len(ID_list)
for input_fn in ID_list:
    print input_fn
    input_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/human/%s.snuN30.bgiN24.pcawgN36.snuN24.dfciN24" % input_fn)
    input_line = input_file.readline().strip()
    

    output_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/human/%s.strictPON.annotated" % input_fn,"w")
    output_file.write(input_line + "\tread_ref;read_var;PON_ref;PON_var;log10OR\n")
    input_line = input_file.readline().strip()
    while input_line:
        input_split = input_line.split('\t')
        
        input_PON_ref = 0
        input_PON_var = 0
        if input_split[30].split(';')[0] == 'NA':
            'blank'
        else:
            input_PON_ref = input_PON_ref + int(input_split[30].split(';')[0])
            input_PON_var = input_PON_var + int(input_split[30].split(';')[2])
        if input_split[-5].split(';')[0] == 'NA':
            'blank'
        else:
            input_PON_ref = input_PON_ref + int(input_split[-5].split(';')[0])
            input_PON_var = input_PON_var + int(input_split[-5].split(';')[2])
        if input_split[-4].split(';')[0] == 'NA':
            'blank'
        else:
            input_PON_ref = input_PON_ref + int(input_split[-4].split(';')[0])  
            input_PON_var = input_PON_var + int(input_split[-4].split(';')[2])
        if input_split[-3].split(';')[0] == 'NA':
            'blank'
        else:
            input_PON_ref = input_PON_ref + int(input_split[-3].split(';')[0]) 
            input_PON_var = input_PON_var + int(input_split[-3].split(';')[2])
        if input_split[-2].split(';')[0] == 'NA':
            'blank'
        else:
            input_PON_ref = input_PON_ref + int(input_split[-2].split(';')[0]) 
            input_PON_var = input_PON_var + int(input_split[-2].split(';')[2])
        if input_split[-1].split(';')[0] == 'NA':
            'blank'
        else:
            input_PON_ref = input_PON_ref + int(input_split[-1].split(';')[0]) 
            input_PON_var = input_PON_var + int(input_split[-1].split(';')[1])            
            
        if input_PON_ref == 0:
            input_PON_ref = 9999
        input_OR = (float(input_PON_var)*float(input_split[29].split(';')[9])) / (float(input_PON_ref)*float(input_split[29].split(';')[10]))
        #print input_OR
        output_file.write(input_line + "\t%s;%s;%s;%s;%s\n" %(input_split[29].split(';')[9],input_split[29].split(';')[10],input_PON_ref,input_PON_var,round(math.log10(input_OR+0.000001),3)))
        
        input_line = input_file.readline().strip()
    output_file.close()
print 'THE END'

30
HBIR2-1.true.vcf
hs_study5_colon_control_HCNor2_SC6.true.vcf
hs_study5_colon_control_HCNor1_SC5.true.vcf
hs_study5_colon_control_HCNor1_SC4.true.vcf
hs_study5_colon_control_HCNor2_SC4.true.vcf
hs_study5_colon_IR_HCIR1_SC2.true.vcf
hs_study5_colon_control_HCNor2_SC3.true.vcf
hs_study5_colon_IR_HCIR1_SC6.true.vcf
hs_study5_colon_control_HCNor2_SC1.true.vcf
hs_study5_colon_control_HCNor2_SC5.true.vcf
HBIR1-1.true.vcf
HBIR1-2.true.vcf
hs_study5_colon_IR_HCIR1_SC1.true.vcf
hs_study5_colon_control_HCNor2_SC2.true.vcf
HBIR2-4.true.vcf
HBIR2-1.true.clonal.vcf
hs_study5_colon_control_HCNor2_SC6.true.clonal.vcf
hs_study5_colon_control_HCNor1_SC5.true.clonal.vcf
hs_study5_colon_control_HCNor1_SC4.true.clonal.vcf
hs_study5_colon_control_HCNor2_SC4.true.clonal.vcf
hs_study5_colon_IR_HCIR1_SC2.true.clonal.vcf
hs_study5_colon_control_HCNor2_SC3.true.clonal.vcf
hs_study5_colon_IR_HCIR1_SC6.true.clonal.vcf
hs_study5_colon_control_HCNor2_SC1.true.clonal.vcf
hs_study5_colon_control_HCNor2_SC5.true.clo

In [33]:
### RIS_SNUH indel strict selection

import math

#ID_list = ['15-495-T', '15-1494T_mem', '15-1034-T', '18_1360T', '19-1621T', '18-2419T', '18_1277T', 'S160041844_T_new', '19-1546T', 'S14_54949_T', 'S15_40859_T', 'S17_11918T', 'S17_50179_T', 'S15_40859_17_T_new', 'S17_11918T_re', 'S12_10269_T']
#ID_list = ['HBIR2-1.true.vcf', 'hs_study5_colon_control_HCNor2_SC6.true.vcf', 'hs_study5_colon_control_HCNor1_SC5.true.vcf', 'hs_study5_colon_control_HCNor1_SC4.true.vcf', 'hs_study5_colon_control_HCNor2_SC4.true.vcf', 'hs_study5_colon_IR_HCIR1_SC2.true.vcf', 'hs_study5_colon_control_HCNor2_SC3.true.vcf', 'hs_study5_colon_IR_HCIR1_SC6.true.vcf', 'hs_study5_colon_control_HCNor2_SC1.true.vcf', 'hs_study5_colon_control_HCNor2_SC5.true.vcf', 'HBIR1-1.true.vcf', 'HBIR1-2.true.vcf', 'hs_study5_colon_IR_HCIR1_SC1.true.vcf', 'hs_study5_colon_control_HCNor2_SC2.true.vcf', 'HBIR2-4.true.vcf', 'HBIR2-1.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC6.true.clonal.vcf', 'hs_study5_colon_control_HCNor1_SC5.true.clonal.vcf', 'hs_study5_colon_control_HCNor1_SC4.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC4.true.clonal.vcf', 'hs_study5_colon_IR_HCIR1_SC2.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC3.true.clonal.vcf', 'hs_study5_colon_IR_HCIR1_SC6.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC1.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC5.true.clonal.vcf', 'HBIR1-1.true.clonal.vcf', 'HBIR1-2.true.clonal.vcf', 'hs_study5_colon_IR_HCIR1_SC1.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC2.true.clonal.vcf', 'HBIR2-4.true.clonal.vcf']
import os, math

temp_list = os.listdir("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/")
file_list = []
ID_list = []
for ll in temp_list:
    if '.true.strictPON.annotated' in ll or '.true.clonal.strictPON.annotated' in ll:
        file_list.append(ll)

print len(file_list)
for input_fn in file_list:
    print input_fn
    input_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/%s" % input_fn)
    input_line = input_file.readline().strip()
    

    output_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/%s.stricttrue.vcf" % input_fn.replace(".true","").replace(".strictPON.annotated",""),"w")
    output_file.write(input_line + "\n")
    input_line = input_file.readline().strip()
    while input_line:
        input_split = input_line.split('\t')
        try:
            read_var = int(input_split[49].split(';')[1])
            logOR = float(input_split[49].split(';')[4])            
        except:
            read_var = int(input_split[42].split(';')[1])
            logOR = float(input_split[42].split(';')[4])
        
        if read_var > 2 and logOR < -3:
            output_file.write(input_line + "\n")
        input_line = input_file.readline().strip()
    output_file.close()
    input_file.close()
print 'THE END'
        


208
mm_study4_breast_sham_SO3.true.strictPON.annotated
S1N2P16_4-6.true.clonal.strictPON.annotated
S1-4Gy-1.true.strictPON.annotated
mm_study4_colon_sham_SO3.true.strictPON.annotated
2Gy_2_S04_S.true.clonal.strictPON.annotated
mm_study4_pancreas_sham_SO3.true.strictPON.annotated
S1-8Gy-1.true.clonal.strictPON.annotated
S1N2P15_1-4a.true.strictPON.annotated
mm_study4_stomach_IR_SO1.true.strictPON.annotated
mm_study4_fallopian_IR_SO1.true.strictPON.annotated
mm_study4_pancreas_IR_SO1.true.strictPON.annotated
S1N2P15_4-1.true.clonal.strictPON.annotated
S1N2P15_1-1.true.clonal.strictPON.annotated
S1N2P16_4-4.true.strictPON.annotated
Liver_C3SO2.true.strictPON.annotated
S1N2P15_1-2a.true.strictPON.annotated
S1N2P15_4-2.true.strictPON.annotated
mm_study4_fallopian_sham_SO2.true.strictPON.annotated
mm_study4_liver_IR_SO3.true.strictPON.annotated
mm_study4_colon_IR_SO3.true.strictPON.annotated
mm_study4_stomach_sham_SO3.true.strictPON.annotated
B3P16_0Gy_4.true.clonal.strictPON.annotated
mm_st

In [140]:
import math
print math.log10(101)
print 74*45/(3*4723)
print math.log10((74*45/(3*4723)))

2.00432137378
0


ValueError: math domain error

In [146]:
input_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/human/human_organoid_vcf.txt")
input_line = input_file.readline().strip()
input_list = []
while input_line:
    input_list.append(input_line)
    input_line = input_file.readline().strip()
print input_list


['HBIR2-1.true.vcf', 'hs_study5_colon_control_HCNor2_SC6.true.vcf', 'hs_study5_colon_control_HCNor1_SC5.true.vcf', 'hs_study5_colon_control_HCNor1_SC4.true.vcf', 'hs_study5_colon_control_HCNor2_SC4.true.vcf', 'hs_study5_colon_IR_HCIR1_SC2.true.vcf', 'hs_study5_colon_control_HCNor2_SC3.true.vcf', 'hs_study5_colon_IR_HCIR1_SC6.true.vcf', 'hs_study5_colon_control_HCNor2_SC1.true.vcf', 'hs_study5_colon_control_HCNor2_SC5.true.vcf', 'HBIR1-1.true.vcf', 'HBIR1-2.true.vcf', 'hs_study5_colon_IR_HCIR1_SC1.true.vcf', 'hs_study5_colon_control_HCNor2_SC2.true.vcf', 'HBIR2-4.true.vcf', 'HBIR2-1.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC6.true.clonal.vcf', 'hs_study5_colon_control_HCNor1_SC5.true.clonal.vcf', 'hs_study5_colon_control_HCNor1_SC4.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC4.true.clonal.vcf', 'hs_study5_colon_IR_HCIR1_SC2.true.clonal.vcf', 'hs_study5_colon_control_HCNor2_SC3.true.clonal.vcf', 'hs_study5_colon_IR_HCIR1_SC6.true.clonal.vcf', 'hs_study5_colon_control_HC

In [161]:
input_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/human/hs_study5_colon_control_HCNor2_SC6.true.clonal.vcf.strictPON.annotated")
input_line = input_file.readline().strip()
input_line = input_file.readline().strip()
while input_line:
    input_split = input_line.split('\t')[40].split(';')
    input_OR = float(input_split[4])
    if input_OR > -2 and int(input_split[1])>17:
        print input_line
    input_line = input_file.readline().strip()
print 'THE END'

1	203929862	.	G	GA	.	11	PASS	SOMATIC;QSI=74;TQSI=2;NT=ref;QSI_NT=74;TQSI_NT=2;SGT=ref->het;MQ=59.80;MQ0=0;RU=A;RC=9;IC=10;IHP=9;SomaticEVS=15.51	DP:DP2:TAR:TIR:TOR:DP50:FDP50:SUBDP50:BCN50	36:36:34,34:0,0:1,1:37.16:0.90:0.00:0.00	40:40:15,17:20,20:4,3:37.91:1.00:0.00:0.03	PASS	DP=70;SOMATIC;SS=2;SSC=65;GPV=1E0;SPV=2.9622E-7	GT:GQ:DP:RD:AD:FREQ:DP4	0/0:.:36:36:0:0%:16,20,0,0	0/1:.:34:17:17:50%:12,5,13,4	20	17	60.0	60.0	9;45.0;136	14;105.0;141	0;0.0;0;0.0	1;5.0;0;0.0	0;0.0;0;0.0	0.0	0.0	36;0;0.0;1,2.78;0,0.0;0,0.0	.;.;20;20;16;17;4;36;0;16;18;3;0;9;.;A	670;19;3;2;0.45	0	0.0	0.53	T	1150;30;16;5;1.39	908;24;1;1;0.11	1782;36;16;11;0.9	746;24;34;3;4.56	1277;24;14;11;1.1	16;18;6533;94;-1.893
3	189848807	.	CT	C	.	11	PASS	SOMATIC;QSI=72;TQSI=1;NT=ref;QSI_NT=72;TQSI_NT=1;SGT=ref->het;MQ=60.00;MQ0=0;RU=T;RC=2;IC=1;IHP=3;SomaticEVS=17.27	DP:DP2:TAR:TIR:TOR:DP50:FDP50:SUBDP50:BCN50	31:31:31,32:0,0:0,0:29.26:0.00:0.00:0.00	33:33:11,11:22,22:0,0:30.24:0.70:0.00:0.00	PASS	DP=58;SOMATIC;SS=2;SSC=71;GPV

In [23]:
#mouse strict PON annotation
import os, math

temp_list = os.listdir("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/")
file_list = []
ID_list = []
for ll in temp_list:
    if '.vcf.b6_N1.b6_M3.b6_others_somatic.b6_germline.balbc_pancreas.balbc_liver.balbc_others' in ll:
        file_list.append(ll)
        ID_list.append(ll.split('.vcf.')[0])
        
#print len(file_list)

target_list = []
info_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/31_1_PanelOfNormal_SPark/00_9_bamlist_b6_others_somatic_add15.txt")
info_line = info_file.readline().strip()
while info_line:
    info_split = info_line.split('/')[-1].split('.s.')[0]
    if info_split == 'mm_study4_germline_2nd_sham':
        'blank'
    else:
        target_list.append(file_list[ID_list.index(info_split+'.true')])
    try:
        target_list.append(file_list[ID_list.index(info_split+'.true.clonal')])
    except:
        'blank'
        
    info_line = info_file.readline().strip()

print len(target_list)

for input_fn in target_list:
    print input_fn.split('.true')[0]
    input_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/%s" % input_fn)
    input_line = input_file.readline().strip()
    

    output_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/%s.strictPON.annotated" % input_fn.split('.vcf')[0],"w")
    output_file.write(input_line + "\tread_ref;read_var;PON_ref;PON_var;log10OR\n")
    input_line = input_file.readline().strip()
    while input_line:
        input_split = input_line.split('\t')
        
        input_PON_ref = 0
        input_PON_var = 0
        if input_split[-7].split(';')[0] == 'NA':
            'blank'
        else:
            input_PON_ref = input_PON_ref + int(input_split[-7].split(';')[0])
            input_PON_var = input_PON_var + int(input_split[-7].split(';')[2])
            
        if input_split[-6].split(';')[0] == 'NA':
            'blank'
        else:
            input_PON_ref = input_PON_ref + int(input_split[-6].split(';')[0])
            input_PON_var = input_PON_var + int(input_split[-6].split(';')[2])            
            
        #if input_split[-5].split(';')[0] == 'NA':
        #    'blank'
        #else:
        #    input_PON_ref = input_PON_ref + int(input_split[-5].split(';')[0])
        #    input_PON_var = input_PON_var + int(input_split[-5].split(';')[2])
        
        if input_split[-4].split(';')[0] == 'NA':
            'blank'
        else:
            input_PON_ref = input_PON_ref + int(input_split[-4].split(';')[0])  
            input_PON_var = input_PON_var + int(input_split[-4].split(';')[2])
            
        #if input_split[-3].split(';')[0] == 'NA':
        #    'blank'
        #else:
        #    input_PON_ref = input_PON_ref + int(input_split[-3].split(';')[0]) 
        #    input_PON_var = input_PON_var + int(input_split[-3].split(';')[2])
        #if input_split[-2].split(';')[0] == 'NA':
        #    'blank'
        #else:
        #    input_PON_ref = input_PON_ref + int(input_split[-2].split(';')[0]) 
        #    input_PON_var = input_PON_var + int(input_split[-2].split(';')[2])
        
        #if input_split[-1].split(';')[0] == 'NA':
        #    'blank'
        #else:
        #    input_PON_ref = input_PON_ref + int(input_split[-1].split(';')[0]) 
        #    input_PON_var = input_PON_var + int(input_split[-1].split(';')[1])            
            
        if input_PON_ref == 0:
            input_PON_ref = 9999
        input_OR = (float(input_PON_var)*float(input_split[29].split(';')[9])) / (float(input_PON_ref)*float(input_split[29].split(';')[10]))
        #print input_OR
        output_file.write(input_line + "\t%s;%s;%s;%s;%s\n" %(input_split[29].split(';')[9],input_split[29].split(';')[10],input_PON_ref,input_PON_var,round(math.log10(input_OR+0.000001),3)))
        
        input_line = input_file.readline().strip()
    output_file.close()
print 'THE END'

15
mm_study4_lung_sham_SO4
mm_study4_lung_sham_SO5
mm_study4_lung_IR_SO1
mm_study4_pancreas_IR_SO1
mm_study4_lung_sham_SO3
mm_study4_lung_IR_SO2
mm_study4_pancreas_IR_SO2
mm_study4_pancreas_sham_SO1
mm_study4_pancreas_sham_SO2
mm_study4_pancreas_sham_SO3
mm_study4_lung_sham_SO1
mm_study4_lung_sham_SO2
mm_study4_lung_IR_SO3
mm_study4_lung_IR_SO4
mm_study4_pancreas_IR_SO3
THE END


In [25]:
import os, math

temp_list = os.listdir("/home/users/jhyouk/06_mm10_SNUH_radiation/32_5_INDEL_191010/mouse/")
file_list = []
ID_list = []
for ll in temp_list:
    if '.true.strictPON.annotated' in ll or '.true.clonal.strictPON.annotated' in ll:
        file_list.append(ll)
print len(file_list)

208
